In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

def generate_box(obj):
    
    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text)
    xmax = int(obj.find('xmax').text)
    ymax = int(obj.find('ymax').text)
    
    return [xmin, ymin, xmax, ymax]

def generate_label(obj):
    if obj.find('name').text == "with_mask":
        return 1
    elif obj.find('name').text == "mask_weared_incorrect":
        return 2
    return 0

def generate_target(image_id, file): 
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, 'xml')
        objects = soup.find_all('object')

        num_objs = len(objects)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i))
            labels.append(generate_label(i))
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # Tensorise img_id
        img_id = torch.tensor([image_id])
        # Annotation is in dictionary format
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = img_id
        
        return target


In [3]:
image_dir = r"C:\Users\shash\Desktop\Code\VSC\datasets\face_masks\images"
# Directory of the mask images. The names should be same as in the image directory, but
imgs = list(sorted(os.listdir(image_dir)))
imgs

['maksssksksss0.png',
 'maksssksksss1.png',
 'maksssksksss10.png',
 'maksssksksss100.png',
 'maksssksksss101.png',
 'maksssksksss102.png',
 'maksssksksss103.png',
 'maksssksksss104.png',
 'maksssksksss105.png',
 'maksssksksss106.png',
 'maksssksksss107.png',
 'maksssksksss108.png',
 'maksssksksss109.png',
 'maksssksksss11.png',
 'maksssksksss110.png',
 'maksssksksss111.png',
 'maksssksksss112.png',
 'maksssksksss113.png',
 'maksssksksss114.png',
 'maksssksksss115.png',
 'maksssksksss116.png',
 'maksssksksss117.png',
 'maksssksksss118.png',
 'maksssksksss119.png',
 'maksssksksss12.png',
 'maksssksksss120.png',
 'maksssksksss121.png',
 'maksssksksss122.png',
 'maksssksksss123.png',
 'maksssksksss124.png',
 'maksssksksss125.png',
 'maksssksksss126.png',
 'maksssksksss127.png',
 'maksssksksss128.png',
 'maksssksksss129.png',
 'maksssksksss13.png',
 'maksssksksss130.png',
 'maksssksksss131.png',
 'maksssksksss132.png',
 'maksssksksss133.png',
 'maksssksksss134.png',
 'maksssksksss135.png',


In [4]:
labels_dir = r"C:\Users\shash\Desktop\Code\VSC\datasets\face_masks\annotations"
labels = list(sorted(os.listdir(labels_dir)))
labels

['maksssksksss0.xml',
 'maksssksksss1.xml',
 'maksssksksss10.xml',
 'maksssksksss100.xml',
 'maksssksksss101.xml',
 'maksssksksss102.xml',
 'maksssksksss103.xml',
 'maksssksksss104.xml',
 'maksssksksss105.xml',
 'maksssksksss106.xml',
 'maksssksksss107.xml',
 'maksssksksss108.xml',
 'maksssksksss109.xml',
 'maksssksksss11.xml',
 'maksssksksss110.xml',
 'maksssksksss111.xml',
 'maksssksksss112.xml',
 'maksssksksss113.xml',
 'maksssksksss114.xml',
 'maksssksksss115.xml',
 'maksssksksss116.xml',
 'maksssksksss117.xml',
 'maksssksksss118.xml',
 'maksssksksss119.xml',
 'maksssksksss12.xml',
 'maksssksksss120.xml',
 'maksssksksss121.xml',
 'maksssksksss122.xml',
 'maksssksksss123.xml',
 'maksssksksss124.xml',
 'maksssksksss125.xml',
 'maksssksksss126.xml',
 'maksssksksss127.xml',
 'maksssksksss128.xml',
 'maksssksksss129.xml',
 'maksssksksss13.xml',
 'maksssksksss130.xml',
 'maksssksksss131.xml',
 'maksssksksss132.xml',
 'maksssksksss133.xml',
 'maksssksksss134.xml',
 'maksssksksss135.xml',


In [5]:
class MaskDataset(object):
    def __init__(self, transforms):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(image_dir)))
        self.labels = list(sorted(os.listdir(labels_dir)))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = 'maksssksksss'+ str(idx) + '.png'
        file_label = 'maksssksksss'+ str(idx) + '.xml'
        img_path = os.path.join(image_dir, file_image)
        label_path = os.path.join(labels_dir, file_label)
        img = Image.open(img_path).convert("RGB")
        #Generate Label
        target = generate_target(idx, label_path)
        
        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [6]:
data_transform = transforms.Compose([
        transforms.ToTensor(), 
    ])

In [7]:
def collate_fn(batch):
    return tuple(zip(*batch))

dataset = MaskDataset(data_transform)
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=4, collate_fn=collate_fn)

In [8]:
torch.cuda.is_available()

True

# Model

In [9]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [10]:
model = get_model_instance_segmentation(3)

c:\Users\shash\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\shash\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break

FeatureNotFound: Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?

# Train Model

In [ ]:

num_epochs = 25
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(data_loader)

for epoch in range(num_epochs):
    model.train()
    i = 0    
    epoch_loss = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model([imgs[0]], [annotations[0]])
        losses = sum(loss for loss in loss_dict.values())        

        optimizer.zero_grad()
        losses.backward()
        optimizer.step() 
        print(f'\t Iteration: {i}/{len_dataloader}, Loss: {losses}')
        epoch_loss += losses
    print(epoch_loss)


In [ ]:
for imgs, annotations in data_loader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        break

In [ ]:
model.eval()
preds = model(imgs)
preds

# Function to plot image

In [ ]:
def plot_image(img_tensor, annotation):
    
    fig,ax = plt.subplots(1)
    img = img_tensor.cpu().data

    # Display the image
    ax.imshow(img.permute(1, 2, 0))
    
    for box in annotation["boxes"]:
        xmin, ymin, xmax, ymax = box

        # Create a Rectangle patch
        rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()

In [ ]:
print("Prediction")
plot_image(imgs[2], preds[2])
print("Target")
plot_image(imgs[2], annotations[2])

In [ ]:
torch.save(model.state_dict(),'model.pt')

In [ ]:
model2 = get_model_instance_segmentation(3)

# Save Model

In [ ]:
model2.load_state_dict(torch.load('model.pt'))
model2.eval()
model2.to(device)

# Load Model

In [ ]:
pred2 = model2(imgs)

In [ ]:
print("Predict with loaded model")
plot_image(imgs[3], pred2[3])